In [1]:
import sys, os
import pandas as pd
import numpy
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

In [2]:
B = [2, 5, 10, 50, 100, 200]

In [3]:
df = pd.read_csv('dating.csv')
discrete_columns = ['gender', 'race', 'race_o', 'samerace', 'field', 'decision']
all_columns = df.columns.values.tolist()
continuous_valued_columns = [item for item in all_columns if item not in discrete_columns]
print all_columns
print continuous_valued_columns

['gender', 'age', 'age_o', 'race', 'race_o', 'samerace', 'importance_same_race', 'importance_same_religion', 'field', 'pref_o_attractive', 'pref_o_sincere', 'pref_o_intelligence', 'pref_o_funny', 'pref_o_ambitious', 'pref_o_shared_interests', 'attractive_important', 'sincere_important', 'intelligence_important', 'funny_important', 'ambition_important', 'shared_interests_important', 'attractive', 'sincere', 'intelligence', 'funny', 'ambition', 'attractive_partner', 'sincere_partner', 'intelligence_parter', 'funny_partner', 'ambition_partner', 'shared_interests_partner', 'sports', 'tvsports', 'exercise', 'dining', 'museums', 'art', 'hiking', 'gaming', 'clubbing', 'reading', 'tv', 'theater', 'movies', 'concerts', 'music', 'shopping', 'yoga', 'interests_correlate', 'expected_happy_with_sd_people', 'like', 'decision']
['age', 'age_o', 'importance_same_race', 'importance_same_religion', 'pref_o_attractive', 'pref_o_sincere', 'pref_o_intelligence', 'pref_o_funny', 'pref_o_ambitious', 'pref_o_

In [4]:
(row, col) = df.shape
age_range = [18.0, 58.0]
pref_score = [0.0, 1.0]
score = [0.0, 10.0]
corr_range = [-1.00, 1.00]

#bin_N = 5
#bin_seg = [1.000 * i/bin_N for i in range(0, bin_N + 1)]
#print bin_seg

age = ['age', 'age_o']
corr = ['interests_correlate']
preference_scores_of_participant = \
['attractive_important', 'sincere_important', 'intelligence_important',\
 'funny_important', 'ambition_important', 'shared_interests_important']

preference_scores_of_partner = \
['pref_o_attractive', 'pref_o_sincere', 'pref_o_intelligence', \
 'pref_o_funny', 'pref_o_ambitious', 'pref_o_shared_interests']

def get_bin_seg(bin_N):
    bin_seg = [1.000 * i/bin_N for i in range(0, bin_N + 1)]
    continuous_valued_columns_bins = {}

    # Segment the bins
    for field in continuous_valued_columns:
        continuous_valued_columns_bins[field] = []
        if field in age:
            for i in range(0, bin_N):
                continuous_valued_columns_bins[field].append(age_range[0] + bin_seg[i] * (age_range[1] - age_range[0]))
        elif field in corr:
            for i in range(0, bin_N):
                continuous_valued_columns_bins[field].append(corr_range[0] + bin_seg[i] * (corr_range[1] - corr_range[0]))
        elif field in preference_scores_of_participant or field in preference_scores_of_partner:
            for i in range(0, bin_N):
                continuous_valued_columns_bins[field].append(pref_score[0] + bin_seg[i] * (pref_score[1] - pref_score[0]))
        else:
            for i in range(0, bin_N):
                continuous_valued_columns_bins[field].append(score[0] + bin_seg[i] * (score[1] - score[0]))
    
    
    #print continuous_valued_columns_bins
    #print len(continuous_valued_columns_bins)
    return continuous_valued_columns_bins

In [5]:
def seg_data(bin_N, continuous_valued_columns_bins):
    # Dictionary of the numbers ine ach bin
    continuous_valued_columns_seg = {}
    # Initalize the dict
    for field in continuous_valued_columns:
        continuous_valued_columns_seg[field] = [0 for i in range(bin_N)]
    
    for i in range(row):
        for field in continuous_valued_columns:
            # Find the bin
            for j in range(0, bin_N):
                # Corner Case
                if j == 0:
                    if continuous_valued_columns_bins[field][j] <= df[field][i] <= continuous_valued_columns_bins[field][j + 1]:
                        df.loc[i,field] = int(j)
                        continuous_valued_columns_seg[field][j] += 1
                        break
                elif j == bin_N - 1:
                    if continuous_valued_columns_bins[field][j] < df[field][i]:
                        df.loc[i,field] = int(j)
                        continuous_valued_columns_seg[field][j] += 1
                        break
                else:
                    if continuous_valued_columns_bins[field][j] < df[field][i] <= continuous_valued_columns_bins[field][j + 1]:
                        df.loc[i,field] = int(j)
                        continuous_valued_columns_seg[field][j] += 1
                        break

In [6]:
def nbc(t_frac, df_train, df_test):
    #df = pd.read_csv(TRAINING_SET).sample(frac=t_frac, random_state=47)
    #df_test = pd.read_csv(TEST_SET).sample(frac=t_frac, random_state=47)
    attr_list = list(df_train[df_train.columns.difference(['decision'])])
    dict_table ={}

    # Labels
    dict_labels = {}
    dict_labels['no'] = len(df_train[df_train['decision'] == 0])
    dict_labels['yes'] = len(df_train[df_train['decision'] == 1])
    dict_table['decision'] = dict_labels
    
    # Attributes in discrete_columns
    for attr in attr_list:
        dict_attr = {}
        attr_bin = max(df_train[attr].max(), df_test[attr].max())
        dict_attr['no'] = [1 for i in range(attr_bin + 1)]
        dict_attr['yes'] = [1 for i in range(attr_bin + 1)]
        
        for i in range(attr_bin+1):
            dict_attr['no'][i] += len(df_train[(df_train[attr] == i) & (df_train['decision'] == 0)])
            dict_attr['yes'][i] += len(df_train[(df_train[attr] == i) & (df_train['decision'] == 1)])

        dict_table[attr] = dict_attr
        
    return dict_table

In [7]:
for bin_N in B:
    df = pd.read_csv('dating.csv')
    continuous_valued_columns_bins = get_bin_seg(bin_N)
    seg_data(bin_N, continuous_valued_columns_bins)
    
    df = df.astype('int64')
    
    print df
    df_test = df.sample(frac=0.2, random_state=47)
    #print df_test.index
    # Subtract 
    df_train = df[~df.index.isin(df_test.index)]
    #print df_train.index
    
    dict_table = nbc(1, df_train, df_train)

    # Accuracy on training data
    #df = pd.read_csv(TRAINING_SET).sample(frac=1, random_state=47)
    (row, col) = df_train.shape
    attr_list = list(df_train[df_train.columns.difference(['decision'])])
    
    neg_num = len(df_train[df_train['decision'] == 0])
    pos_num = len(df_train[df_train['decision'] == 1])
    
    correct = 0
    for i in range(row):
        pd_pos = 1.0 * dict_table['decision']['yes']/row
        pd_neg = 1.0 * dict_table['decision']['no']/row
        for attr in attr_list:
            pd_pos *= 1.0 * dict_table[attr]['yes'][df_train[attr][i]]/pos_num
            pd_neg *= 1.0 * dict_table[attr]['no'][df_train[attr][i]]/neg_num
        
        res = np.argmax([1.0 * pd_neg, 1.0 * pd_pos])
        if res == df_train['decision'][i]:
            correct += 1
    #print correct
    training_accuracy = 1.0 * correct/row
    print 'Bin size:', bin_N
    print 'Training Accuracy:', '%.2f' % training_accuracy

      gender  age  age_o  race  race_o  samerace  importance_same_race  \
0          0    0      0     0       2         0                     0   
1          0    0      0     0       2         0                     0   
2          0    0      0     0       2         0                     0   
3          0    0      0     0       3         0                     0   
4          0    0      0     0       2         0                     0   
5          0    0      0     0       2         0                     0   
6          0    0      0     0       2         0                     0   
7          0    0      0     0       2         0                     0   
8          0    0      0     2       2         1                     0   
9          0    0      0     2       2         1                     0   
10         0    0      0     2       0         0                     0   
11         0    0      0     2       2         1                     0   
12         0    0      0     2       3

KeyError: 7